<a href="https://colab.research.google.com/github/jazaineam1/BigDataMINE2023/blob/main/Copia_de_7_Pyspark_y_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Spark intro 2***


## ***Universidad Externado de Colombia***

>## ***Maestría en Inteligencia de Negocios***
![Imágen1](https://www.uexternado.edu.co/wp-content/uploads/2020/07/logo-uec.png)

jazaineam@unal.edu.co


>## ***Big Data.***
>## ***Docente: Antonino Zainea Maya.***

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=091465fdb30891ef8860670510bcf9f701a8ad899bf2eaa9039dfb2681ead364
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import os

from pyspark.sql import SparkSession
from pyspark.ml  import Pipeline
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
# from pyspark.ml.feature import StringIndexer, VectorAssembler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.feature import QuantileDiscretizer

# **1. Información de Secop II Contratos** <a class="anchor" id="1"></a>


En primer lugar, debemos crear un contenedor de Spark llamando a SparkSession. Este paso es necesario antes de realizar cualquier acción.

In [3]:
# Create SparkSession
spark = SparkSession.builder.appName('Ejemplo Secop II').getOrCreate()

Descarguemos la información correspondiente a los contratos del Secop II y contiene las siguientes variables:

**Nombre Entidad:** Nombre de la entidad gubernamental o institución pública que realiza la contratación.

**Nit Entidad:** Número de Identificación Tributaria de la entidad contratante.

**Departamento:** Departamento de Colombia donde se ubica la entidad contratante.

**Ciudad:**Ciudad donde se ubica la entidad contratante.

**Sector:** Sector gubernamental al que pertenece la entidad contratante (ejemplo: salud, educación, defensa).

**Estado Contrato:** Estado actual del contrato (ejemplo: en ejecución, finalizado, liquidado).

**Descripción del Proceso:** Breve descripción del proceso de contratación o del objeto a contratar.

**Tipo de Contrato**: Clasificación del contrato según su naturaleza (ejemplo: obra, suministro, servicio).

**Modalidad de Contratación:** Método utilizado para seleccionar al proveedor (ejemplo: licitación pública, contratación directa).

**Fecha de Firma:** Fecha en la que se firma el contrato entre la entidad y el proveedor.

**Fecha de Inicio del Contrato**: Fecha en la que comienza la ejecución del contrato.

**Fecha de Fin del Contrato:** Fecha prevista para la finalización del contrato.

**Documento Proveedor:** Número de identificación o NIT del proveedor o contratista adjudicado.

**Proveedor Adjudicado:** Nombre del proveedor o contratista al que se le adjudica el contrato.

**Es Pyme:** Indica si el proveedor adjudicado es una pequeña o mediana empresa (Pyme).

**Valor del Contrato:** Monto económico total del contrato.

**URLProceso:** Enlace a la página web donde se pueden consultar los detalles del proceso de contratación.

**Nacionalidad Representante Legal:** Nacionalidad del representante legal del proveedor adjudicado.

**Género Representante Legal:** Género (masculino, femenino, otro) del representante legal del proveedor.

**Objeto del Contrato:** Descripción detallada del servicio, bien o obra objeto del contrato.

In [4]:
# Descargar el archivo ZIP de la carpeta
!wget https://github.com/jazaineam1/BigDataMINE2023/archive/refs/heads/main.zip -O repo.zip

# Extraer solo la carpeta que necesitas
!unzip -j repo.zip "BigDataMINE2023-main/data/Secop_II_Contratos/*" -d SecopII_data

# Eliminar el archivo ZIP
!rm repo.zip


--2024-02-10 06:50:49--  https://github.com/jazaineam1/BigDataMINE2023/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/jazaineam1/BigDataMINE2023/zip/refs/heads/main [following]
--2024-02-10 06:50:50--  https://codeload.github.com/jazaineam1/BigDataMINE2023/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘repo.zip’

repo.zip                [             <=>    ]   1.21G  15.9MB/s    in 86s     

2024-02-10 06:52:16 (14.4 MB/s) - ‘repo.zip’ saved [1305113906]

Archive:  repo.zip
1909eef3aca383772d42c9b2dfef65728d26b242
  inflating: SecopII_data/Secop_ii_0.parquet  
  inflating: 

In [22]:
# Directorio que contiene los archivos Parquet
directory = "SecopII_data/"

# Carga los archivos Parquet en un DataFrame
df = spark.read.parquet(directory)

In [6]:
df.show(5)

+--------------------+-----------+--------------------+------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+--------------------+--------------------------------+--------------------------+--------------------+
|      Nombre Entidad|Nit Entidad|        Departamento|Ciudad|              Sector|Estado Contrato|Descripcion del Proceso|    Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|          URLProceso|Nacionalidad Representante Legal|Género Representante Legal| Objeto del Contrato|
+--------------------+-----------+--------------------+------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------

In [7]:

df.limit(3).toPandas()

,Nombre Entidad,Nit Entidad,Departamento,Ciudad,Sector,Estado Contrato,Descripcion del Proceso,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Documento Proveedor,Proveedor Adjudicado,Es Pyme,Valor del Contrato,URLProceso,Nacionalidad Representante Legal,Género Representante Legal,Objeto del Contrato
0,SENADO DE LA REPÚBLICA,"899,999,103",Distrito Capital de Bogotá,Bogotá,No aplica/No pertenece,Modificado,El CONTRATISTA SE OBLIGA A LA PRESTACIÓN DE SE...,Prestación de servicios,Contratación directa,03/17/2023,03/21/2023,06/30/2023,4646766,JORGE DARIO CAMPO BOMBA,No,"17,500,000",https://community.secop.gov.co/Public/Tenderin...,CO,No Definido,El CONTRATISTA SE OBLIGA A LA PRESTACIÓN DE SE...
1,MUNICIPIO DE SAN JOSE DE CUCUTA,"890,501,434",Norte de Santander,Cúcuta,Servicio Público,Modificado,2023B-58 PRESTAR LOS SERVICIOS OPERATIVOS Y AS...,Prestación de servicios,Contratación directa,02/02/2023,02/02/2023,11/02/2023,8829339,OLMER ENRIQUE AGUIRRE ROJAS,No,"16,200,000",https://community.secop.gov.co/Public/Tenderin...,CO,No Definido,2023B-58 PRESTAR LOS SERVICIOS OPERATIVOS Y AS...
2,DANE - DIRECCION TERRITORIAL CENTRO,"899,999,027",Distrito Capital de Bogotá,Bogotá,Información Estadística,Cerrado,CENSO_ECONOMICO_2021_DIG_TH_TU_ OT_MN Prestaci...,Prestación de servicios,Contratación directa,01/20/2021,01/21/2021,04/21/2021,4153397,fermin emigdio estrada santos,No,"10,493,431",https://community.secop.gov.co/Public/Tenderin...,CO,No Definido,CENSO_ECONOMICO_2021_DIG_TH_TU_ OT_MN Prestaci...


In [ ]:
df

DataFrame[Entidad: string, Nit Entidad: bigint, Departamento Entidad: string, Ciudad Entidad: string, OrdenEntidad: string, Entidad Centralizada: string, ID del Proceso: string, Referencia del Proceso: string, PCI: string, ID del Portafolio: string, Nombre del Procedimiento: string, Descripción del Procedimiento: string, Fase: string, Fecha de Publicacion del Proceso: string, Fecha de Ultima Publicación: string, Fecha de Publicacion (Fase Planeacion Precalificacion): double, Fecha de Publicacion (Fase Seleccion Precalificacion): double, Fecha de Publicacion (Manifestacion de Interes): string, Fecha de Publicacion (Fase Borrador): string, Fecha de Publicacion (Fase Seleccion): string, Precio Base: bigint, Modalidad de Contratacion: string, Justificación Modalidad de Contratación: string, Duracion: bigint, Unidad de Duracion: string, Fecha de Recepcion de Respuestas: string, Fecha de Apertura de Respuesta: string, Fecha de Apertura Efectiva: string, Ciudad de la Unidad de Contratación: s

Nuestra tarea consiste en predecir y devolver una lista de recomendaciones de películas para un usuario en particular.

Usamos el método `printSchema()` para obtener una vista rápida de los tipos de datos de las características en nuestros datos. Esto nos permite comprender la estructura de los datos y verificar cómo están organizadas las columnas en el conjunto de datos.

In [8]:
df.printSchema()

root
 |-- Nombre Entidad: string (nullable = true)
 |-- Nit Entidad: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Estado Contrato: string (nullable = true)
 |-- Descripcion del Proceso: string (nullable = true)
 |-- Tipo de Contrato: string (nullable = true)
 |-- Modalidad de Contratacion: string (nullable = true)
 |-- Fecha de Firma: string (nullable = true)
 |-- Fecha de Inicio del Contrato: string (nullable = true)
 |-- Fecha de Fin del Contrato: string (nullable = true)
 |-- Documento Proveedor: string (nullable = true)
 |-- Proveedor Adjudicado: string (nullable = true)
 |-- Es Pyme: string (nullable = true)
 |-- Valor del Contrato: string (nullable = true)
 |-- URLProceso: string (nullable = true)
 |-- Nacionalidad Representante Legal: string (nullable = true)
 |-- Género Representante Legal: string (nullable = true)
 |-- Objeto del Contrato: string (nullable = true)



In [9]:
df.limit(10000).describe().show()

+-------+--------------------+-------------+------------+---------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+-------------------------+--------------------+--------------------+-------+------------------+--------------------+--------------------------------+--------------------------+--------------------+
|summary|      Nombre Entidad|  Nit Entidad|Departamento|   Ciudad|              Sector|Estado Contrato|Descripcion del Proceso|    Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato| Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|          URLProceso|Nacionalidad Representante Legal|Género Representante Legal| Objeto del Contrato|
+-------+--------------------+-------------+------------+---------+--------------------+---------------+-----------------------+--------------------+-----------------

In [51]:
df2=df.filter((df["Nit Entidad"] == "899,999,027") & (df['Estado Contrato']=='Activo'))
df2.show()


+--------------------+-----------+--------------------+-----------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+--------------------+--------------------------------+--------------------------+--------------------+
|      Nombre Entidad|Nit Entidad|        Departamento|     Ciudad|              Sector|Estado Contrato|Descripcion del Proceso|    Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|          URLProceso|Nacionalidad Representante Legal|Género Representante Legal| Objeto del Contrato|
+--------------------+-----------+--------------------+-----------+--------------------+---------------+-----------------------+--------------------+-------------------

In [28]:
df2.select('Nombre Entidad').show()

+--------------------+
|      Nombre Entidad|
+--------------------+
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DANE - DIRECCION ...|
|DANE - DIRECCION ...|
|DEPARTAMENTO ADMI...|
|DANE - DIRECCION ...|
|DANE - DIRECCION ...|
|DEPARTAMENTO ADMI...|
|DANE CENTRO OCCID...|
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DEPARTAMENTO ADMI...|
|DANE - DIRECCION ...|
|DANE CENTRO OCCID...|
|DANE - DIRECCION ...|
|DEPARTAMENTO ADMI...|
|DANE - DIRECCION ...|
+--------------------+
only showing top 20 rows



In [26]:
print(df2.count(),len(df2.columns))

5414 20


## Comprobación de valores nulos en PySpark:

- `isnan()` es una función del paquete `pyspark.sql.functions` y debemos especificar qué columna queremos utilizar como argumento de la función.
- `isNull()` pertenece al paquete `pyspark.sql.Column` y se utiliza para verificar el estado nulo de una columna en particular.

Para verificar valores nulos en PySpark, utilizamos ambas funciones mencionadas anteriormente.

In [44]:

df2.select(count(when(col('Es Pyme').isNull(),1)).alias('Nulos es pyme')).show()


+-------------+
|Nulos es pyme|
+-------------+
|            0|
+-------------+



In [45]:
df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+--------------+-----------+------------+------+------+---------------+-----------------------+----------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+----------+--------------------------------+--------------------------+-------------------+
|Nombre Entidad|Nit Entidad|Departamento|Ciudad|Sector|Estado Contrato|Descripcion del Proceso|Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|URLProceso|Nacionalidad Representante Legal|Género Representante Legal|Objeto del Contrato|
+--------------+-----------+------------+------+------+---------------+-----------------------+----------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+-------

In [58]:
df2=df2.drop("URLProceso")


In [60]:
df2.columns

['Nombre Entidad',
 'Nit Entidad',
 'Departamento',
 'Ciudad',
 'Sector',
 'Estado Contrato',
 'Descripcion del Proceso',
 'Tipo de Contrato',
 'Modalidad de Contratacion',
 'Fecha de Firma',
 'Fecha de Inicio del Contrato',
 'Fecha de Fin del Contrato',
 'Documento Proveedor',
 'Proveedor Adjudicado',
 'Es Pyme',
 'Valor del Contrato',
 'Nacionalidad Representante Legal',
 'Género Representante Legal',
 'Objeto del Contrato']

Para reemplazar estos valores "," en la variable Valor del contrato.

En la característica 'Name' (nombre), podemos ver que los nombres tienen una salutación como Sr. o Sra. Por lo tanto, podemos asignar los valores promedio de Sr. y Sra. a los grupos respectivos.

**Utilizando la expresión regular "[A-Za-z]+)." extraemos las iniciales del nombre. Esta expresión busca cadenas que se encuentran entre A-Z o a-z y están seguidas por un punto (.).**

In [61]:
df2 = df2.withColumn('Valor del Contrato', regexp_replace(col('Valor del Contrato'), ',', ''))
df2.limit(3).show()


+--------------------+-----------+--------------------+------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+--------------------------------+--------------------------+--------------------+
|      Nombre Entidad|Nit Entidad|        Departamento|Ciudad|              Sector|Estado Contrato|Descripcion del Proceso|    Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|Nacionalidad Representante Legal|Género Representante Legal| Objeto del Contrato|
+--------------------+-----------+--------------------+------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+------

In [63]:
df2.printSchema()

root
 |-- Nombre Entidad: string (nullable = true)
 |-- Nit Entidad: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Estado Contrato: string (nullable = true)
 |-- Descripcion del Proceso: string (nullable = true)
 |-- Tipo de Contrato: string (nullable = true)
 |-- Modalidad de Contratacion: string (nullable = true)
 |-- Fecha de Firma: string (nullable = true)
 |-- Fecha de Inicio del Contrato: string (nullable = true)
 |-- Fecha de Fin del Contrato: string (nullable = true)
 |-- Documento Proveedor: string (nullable = true)
 |-- Proveedor Adjudicado: string (nullable = true)
 |-- Es Pyme: string (nullable = true)
 |-- Valor del Contrato: string (nullable = true)
 |-- Nacionalidad Representante Legal: string (nullable = true)
 |-- Género Representante Legal: string (nullable = true)
 |-- Objeto del Contrato: string (nullable = true)



In [66]:
df2 = df2.withColumn('Valor del Contrato', df2['Valor del Contrato'].cast('float'))

In [68]:
df2.select('Valor del Contrato')

DataFrame[Valor del Contrato: float]

In [71]:
# Embarked columns has two missing values, let check it
df2.groupBy("Departamento").agg(sum('Valor del Contrato')/1000000).show()

+--------------------+-----------------------------------+
|        Departamento|(sum(Valor del Contrato) / 1000000)|
+--------------------+-----------------------------------+
|           Antioquia|                        3979.586118|
|           Atlántico|                        6212.848891|
|Distrito Capital ...|                         83588.7334|
|     Valle del Cauca|                        4941.758931|
|              Caldas|                        3147.685367|
|           Santander|                            23.6555|
+--------------------+-----------------------------------+



Usamos `PysparkDataFrame.na.fill()` para llenar un valor en una columna específica.

In [72]:
# As we can see, Majority Passengers boarded from "S". We can assign missing value with "S"
df2 = df2.na.fill({"Fecha de Inicio del Contrato" : 'Sin Información'})
# Check again the missing value status
df2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+--------------+-----------+------------+------+------+---------------+-----------------------+----------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+--------------------------------+--------------------------+-------------------+
|Nombre Entidad|Nit Entidad|Departamento|Ciudad|Sector|Estado Contrato|Descripcion del Proceso|Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|Nacionalidad Representante Legal|Género Representante Legal|Objeto del Contrato|
+--------------+-----------+------------+------+------+---------------+-----------------------+----------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+----------

In [74]:
df2 = df2.withColumn("Valor del Contrato M",col('Valor del Contrato')/1000000)
df2 = df2.withColumn('Variable nueva',lit('Ejemplo'))
df2 = df2.withColumn("Millonario?",when(df2["Valor del Contrato"] >1, 'Si').otherwise('No'))
df2.show()


+--------------------+-----------+--------------------+-----------+--------------------+---------------+-----------------------+--------------------+-------------------------+--------------+----------------------------+-------------------------+-------------------+--------------------+-------+------------------+--------------------------------+--------------------------+--------------------+--------------------+--------------+-----------+
|      Nombre Entidad|Nit Entidad|        Departamento|     Ciudad|              Sector|Estado Contrato|Descripcion del Proceso|    Tipo de Contrato|Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Documento Proveedor|Proveedor Adjudicado|Es Pyme|Valor del Contrato|Nacionalidad Representante Legal|Género Representante Legal| Objeto del Contrato|Valor del Contrato M|Variable nueva|Millonario?|
+--------------------+-----------+--------------------+-----------+--------------------+---------------+----------

In [78]:
df2.filter(df2['Millonario?']=='No').count()

10

### Ejercicio avance 1

Con los datos disponibles en su proyecto, realice los siguientes pasos:

1. Cree un nuevo proyecto en Google Cloud Platform (GCP).
2. Establezca un nuevo "bucket" en GCP y almacene allí la información de su proyecto.

Si su objetivo es desarrollar un modelo de predicción de aprendizaje automático, siga estos pasos adicionales:

3.1. Construya una tabla en BigQuery utilizando los datos de su proyecto.

3.2. Configure un clúster en Dataproc y realice la limpieza de datos usando Spark.


